# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Eikonal equations, acceleration and reproducibility
## Chapter : Riemannian metrics

In this notebook, we solve Riemannian eikonal equations on the CPU and the GPU, and check that they produce consistent results.

**GPU performance** GPUs are massively parallel machines, which efficiently exploit cache locality. Hence they are at their advantage with :
* Large problem instances, which are embarassingly parallel
* Moderate anisotropy, so that the numerical scheme stncils are not too wide

In [81]:
large_instances = False # True favors the GPU code (CPU times may become a big long.)
strong_anisotropy = True # True favors the CPU code 
anisotropy_bound = 10. if strong_anisotropy else 4. # Ratio between the fastest and the smallest velocity at any given point

[**Summary**](Summary.ipynb) of volume GPU accelerated methods, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Two dimensions](#1.-Two-dimensions)
    * [1.1 Isotropic metric](#1.1-Isotropic-metric)
    * [1.2 Smooth anisotropic metric](#1.2-Smooth-anisotropic-metric)
  * [2. Three dimensions](#2.-Three-dimensions)
    * [2.1 Smooth anisotropic metric](#2.1-Smooth-anisotropic-metric)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research.
The author would like to acknowledge fruitful informal discussions with L. Gayraud on the 
topic of GPU coding and optimization.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [2]:
import sys; sys.path.insert(0,"..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Riemann_Repro','GPU'))

In [3]:
import cupy as cp
import numpy as np
import itertools
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [4]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
import agd.AutomaticDifferentiation.cupy_generic as cugen

from Notebooks_GPU.ExportedCode.Isotropic_Repro import RunCompare

In [5]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global HFMUtils,ad,cugen,RunGPU,RunSmart,Metrics
    HFMUtils,ad,cugen,Metrics = rreload([HFMUtils,ad,cugen,Metrics],"../..")    
    HFMUtils.dictIn.RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

In [6]:
cp = ad.functional.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
plt = ad.functional.decorate_module_functions(plt,cugen.cupy_get_args)
HFMUtils.dictIn.RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

## 1. Two dimensions

### 1.1 Isotropic metric

In [9]:
n=4000 if large_instances else 1000
hfmIn = HFMUtils.dictIn({
    'model':'Riemann2',
    'metric':Metrics.Riemann.from_cast(Metrics.Isotropic(cp.array(1.),vdim=2)),
    'seeds':cp.array([[0.5,0.5]]),
    'exportValues':1,
#    'bound_active_blocks':True,
    'traits':{
        'niter_i':24,'shape_i':(12,12), # Best
    }
})
hfmIn.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)

Casting output of function array from float64 to float32
Casting output of function array from float64 to float32


In [10]:
_,cpuOut = RunCompare(hfmIn,check=1e-5)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.06599617004394531 seconds,  and 86 iterations.
Post-Processing
--- gpu done, turning to cpu ---
Field verbosity defaults to 1
Field order defaults to 1
Field seedRadius defaults to 0
Fast marching solver completed in 0.818 s.
Solver time (s). GPU : 0.06599617004394531, CPU : 1.508. Device acceleration : 22.84981081471634
Max |gpuValues-cpuValues| :  2.8908252719395122e-06


In [14]:
n=200; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)
X = hfmInS.Grid()
hfmInS.update({
    'metric': Metrics.Riemann.from_cast(Metrics.Isotropic(cost,vdim=2)), # Isotropic but non-constant metric
    'verbosity':0,
})

In [16]:
RunCompare(hfmInS,variants='basic')

Solver time (s). GPU : 0.008500099182128906, CPU : 0.055. Device acceleration : 6.47051273420846
Max |gpuValues-cpuValues| :  4.017569794623199e-07

 --- Variant {'multiprecision': True} ---
Solver time (s). GPU : 0.011510610580444336, CPU : 0.05. Device acceleration : 4.343818223244061
Max |gpuValues-cpuValues| :  4.940263631514341e-08

 --- Variant {'seedRadius': 2.0} ---
Solver time (s). GPU : 0.008011579513549805, CPU : 0.049. Device acceleration : 6.116147248757551
Max |gpuValues-cpuValues| :  4.202669309227858e-07

 --- Variant {'seedRadius': 2.0, 'multiprecision': True} ---
Solver time (s). GPU : 0.011472225189208984, CPU : 0.051000000000000004. Device acceleration : 4.445519431397814
Max |gpuValues-cpuValues| :  9.997386174465106e-08


In [17]:
RunCompare(hfmInS,variants='ext',check=0.004)

Solver time (s). GPU : 0.010999679565429688, CPU : 0.049. Device acceleration : 4.454675221085487
Max |gpuValues-cpuValues| :  4.017569794623199e-07

 --- Variant {'multiprecision': True} ---
Solver time (s). GPU : 0.011474132537841797, CPU : 0.05. Device acceleration : 4.35762789344637
Max |gpuValues-cpuValues| :  4.940263631514341e-08

 --- Variant {'seedRadius': 2.0} ---
Solver time (s). GPU : 0.011005878448486328, CPU : 0.051000000000000004. Device acceleration : 4.633887266582904
Max |gpuValues-cpuValues| :  4.202669309227858e-07

 --- Variant {'seedRadius': 2.0, 'multiprecision': True} ---
Solver time (s). GPU : 0.010988473892211914, CPU : 0.05. Device acceleration : 4.550222395799432
Max |gpuValues-cpuValues| :  9.997386174465106e-08

 --- Variant {'factoringRadius': 10.0, 'factoringPointChoice': 'Key'} ---
Solver time (s). GPU : 0.011499881744384766, CPU : 0.053. Device acceleration : 4.608743044325579
Max |gpuValues-cpuValues| :  0.00019995271131245662

 --- Variant {'factorin

### 1.2 Smooth anisotropic metric

In [24]:
n=4000 if large_instances else 1000
hfmIn = HFMUtils.dictIn({
    'model':'Riemann2',
    'seeds':cp.array([[0.,0.]]),
    'exportValues':1,
#    'bound_active_blocks':True,
    'traits':{
        'niter_i':16,'shape_i':(8,8), # Best
    },
})
hfmIn.SetRect([[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)

Casting output of function array from float64 to float32


In [25]:
def height(x): return np.sin(x[0])*np.sin(x[1])
def surface_metric(x,z,mu):
    ndim,shape = x.ndim-1,x.shape[1:]
    x_ad = ad.Dense.identity(constant=x,shape_free=(ndim,))
    tensors = lp.outer_self( z(x_ad).gradient() ) + mu**-2 * fd.as_field(cp.eye(ndim),shape)
    return Metrics.Riemann(tensors)

In [26]:
hfmIn['metric'] = surface_metric(hfmIn.Grid(),height,mu=anisotropy_bound)

Casting output of function eye from float64 to float32


In [27]:
gpuOut,cpuOut = RunCompare(hfmIn,check=False)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 4.2369983196258545 seconds,  and 988 iterations.
Post-Processing
--- gpu done, turning to cpu ---
Field verbosity defaults to 1
Field order defaults to 1
Field seedRadius defaults to 0
Fast marching solver completed in 29.539 s.
Solver time (s). GPU : 4.2369983196258545, CPU : 47.748000000000005. Device acceleration : 11.269298781363775
Max |gpuValues-cpuValues| :  0.00041117211812591314


In [51]:
n=200; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)
hfmInS.update({
    'metric' : surface_metric(hfmInS.Grid(),height,mu=anisotropy_bound), 
    'verbosity':0,
})

Casting output of function eye from float64 to float32


In [53]:
RunCompare(hfmInS,variants=True,tol=1e-5)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.036499738693237305, CPU : 0.094. Device acceleration : 2.575360902992338
Max |gpuValues-cpuValues| :  2.2354527901891075e-05

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.036981821060180664, CPU : 0.09. Device acceleration : 2.4336281291703465
Max |gpuValues-cpuValues| :  3.011527211893572e-05

--- Variant with {'seedRadius': 2.0} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03198885917663574, CPU : 0.093. Device acceleration : 2.907262165445588
Max |gpuValues-cpuValues| :  2.423421811181825e-05

--- Variant with {'seedRadius': 2.0} and {'multiprecision': True} ---
Solver time (s). GPU : 0.03000044822692871, CPU : 0.09. Device acceleration : 2.99995517797681
Max |gpuValues-cpuValues| :  3.0149078184216194e-05

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03549814224243

'\nfor fact,multip in itertools.product(factor_variants,multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip})\n'

Due to the different switching criteria of the second order scheme, we do not have bit consistency in that case. The results are nevertheless quite close. Note also that we do not deactivate the `decreasing` trait here, contrary to the isotropic case, because the scheme often does not converge without it.

**Bottom line.** Second order accuracy for anisotropic metrics on the GPU is very experimental, and not much reliable, at this stage. Further investigation is needed on the matter.

In [55]:
hfmInS.update({
    'seeds':[[0.,1.]],
    'order':2,
})

In [57]:
RunCompare(hfmInS,variants=True,check=0.1)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.0339961051940918, CPU : 0.119. Device acceleration : 3.5004009818360333
Max |gpuValues-cpuValues| :  0.08098238901310739

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.04849743843078613, CPU : 0.11499999999999999. Device acceleration : 2.3712592607158833
Max |gpuValues-cpuValues| :  0.08113533453160104

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03948044776916504, CPU : 0.11499999999999999. Device acceleration : 2.9128342381622407
Max |gpuValues-cpuValues| :  0.08095333137777927

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': True} ---
Solver time (s). GPU : 0.055997371673583984, CPU : 0.11399999999999999. Device acceleration : 2.0358098352279983
Max |gpuValues-cpuValues| :  0.08113533453160104


'\nfor fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip},check=False)\n'

If one removes enforced monotonicity, obtaining the scheme convergence is harder, and requires setting some other parameters carefully and conservatively.
<!---
hfmInS.update({
    'order2_threshold':0.03,
    'verbosity':1,
    'traits':{'decreasing_macro':0,'order2_threshold_weighted_macro':1},
    'metric' : surface_metric(hfmInS.Grid(),height),
    'multiprecision':False,
    'tol':1e-6
})
--->

In [61]:
hfmInS.update({
    'tol':1e-6, # Tolerance for the convergence of the fixed point solver
    'order2_threshold':0.03, # Use first order scheme if second order difference is too large
    'traits':{'decreasing_macro':0}, # Do not enforce monotonicity
})

In [64]:
RunCompare(hfmInS,variants=True,check=0.15)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.040494441986083984, CPU : 0.11399999999999999. Device acceleration : 2.8152011586967016
Max |gpuValues-cpuValues| :  0.13615021364053104

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.03898048400878906, CPU : 0.11499999999999999. Device acceleration : 2.9501942555169545
Max |gpuValues-cpuValues| :  0.13615051166375491

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03999590873718262, CPU : 0.118. Device acceleration : 2.9503017614974216
Max |gpuValues-cpuValues| :  0.13615045205911014

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': True} ---
Solver time (s). GPU : 0.04599928855895996, CPU : 0.11499999999999999. Device acceleration : 2.5000386658719256
Max |gpuValues-cpuValues| :  0.13615051166375491


'\nfor fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip},check=False)\n'

In [ ]:
# TODO : discontinuous metric

## 2. Three dimensions

### 2.1 Smooth anisotropic metric

We generalize the two dimensional test case, although it does not much make geometrical sense anymore: we are computing geodesics in a three dimensional volume viewed as an hypersurface embedded in four dimensional Euclidean space.

In [82]:
n=200 if large_instances else 100
hfmIn = HFMUtils.dictIn({
    'model':'Riemann3',
    'seeds':cp.array([[0.,0.,0.]]),
    'exportValues':1,
#    'tol':5e-3,
#    'multiprecision':True,
#    'bound_active_blocks':True,
#    'nitermax_o':200,
#    'raiseOnNonConvergence':0,
})
hfmIn.SetRect([[-np.pi,np.pi],[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)

Casting output of function array from float64 to float32


In [84]:
def height3(x): return np.sin(x[0])*np.sin(x[1])*np.sin(x[2])

In [85]:
hfmIn['metric'] = surface_metric(hfmIn.Grid(),height3,mu=anisotropy_bound)

Casting output of function eye from float64 to float32


In [86]:
gpuOut,cpuOut = RunCompare(hfmIn,check=1e-4)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
mean_cost_bound 0.037707984
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.1379985809326172 seconds,  and 60 iterations.
Post-Processing
--- gpu done, turning to cpu ---
Field verbosity defaults to 1
Field order defaults to 1
Field seedRadius defaults to 0
Fast marching solver completed in 6.511 s.
Solver time (s). GPU : 0.1379985809326172, CPU : 9.773. Device acceleration : 70.81956882420423
Max |gpuValues-cpuValues| :  8.216733560773015e-06


In [88]:
n=20; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[-np.pi,np.pi],[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)
hfmInS.update({
    'metric' : surface_metric(hfmInS.Grid(),height,mu=anisotropy_bound), 
    'verbosity':0,
})

Casting output of function eye from float64 to float32


In [89]:
RunCompare(hfmInS,variants='basic')

mean_cost_bound 0.21939948
Solver time (s). GPU : 0.006486415863037109, CPU : 0.038. Device acceleration : 5.858397118282731
Max |gpuValues-cpuValues| :  2.67317871505135e-07

 --- Variant {'multiprecision': True} ---
mean_cost_bound 0.21939948
Solver time (s). GPU : 0.009496450424194336, CPU : 0.037000000000000005. Device acceleration : 3.896192613793277
Max |gpuValues-cpuValues| :  4.1609422674060426e-07

 --- Variant {'seedRadius': 2.0} ---
mean_cost_bound 0.21939948
Solver time (s). GPU : 0.007990121841430664, CPU : 0.036000000000000004. Device acceleration : 4.505563333631725
Max |gpuValues-cpuValues| :  2.0110513232474148e-07

 --- Variant {'seedRadius': 2.0, 'multiprecision': True} ---
mean_cost_bound 0.21939948
Solver time (s). GPU : 0.008496761322021484, CPU : 0.037000000000000005. Device acceleration : 4.354600370391156
Max |gpuValues-cpuValues| :  4.477440634920171e-07


Due to the different switching criteria of the second order scheme, we do not have bit consistency in that case. The results are nevertheless quite close.

In [91]:
RunCompare(hfmInS,variants='ext',check=0.1)

mean_cost_bound 0.21939948
Solver time (s). GPU : 0.009492874145507812, CPU : 0.037000000000000005. Device acceleration : 3.897660438014869
Max |gpuValues-cpuValues| :  2.67317871505135e-07

 --- Variant {'multiprecision': True} ---
mean_cost_bound 0.21939948
Solver time (s). GPU : 0.009001016616821289, CPU : 0.037000000000000005. Device acceleration : 4.11064678303711
Max |gpuValues-cpuValues| :  4.1609422674060426e-07

 --- Variant {'seedRadius': 2.0} ---
mean_cost_bound 0.21939948
Solver time (s). GPU : 0.007001161575317383, CPU : 0.037000000000000005. Device acceleration : 5.284837323344118
Max |gpuValues-cpuValues| :  2.0110513232474148e-07

 --- Variant {'seedRadius': 2.0, 'multiprecision': True} ---
mean_cost_bound 0.21939948
Solver time (s). GPU : 0.00850224494934082, CPU : 0.035. Device acceleration : 4.116559827262276
Max |gpuValues-cpuValues| :  4.477440634920171e-07

 --- Variant {'factoringRadius': 10.0, 'factoringPointChoice': 'Key'} ---
mean_cost_bound 0.21939948
Solver 